In [13]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [14]:
import re

with open('input.txt', 'r') as f:
    instructions = [list(re.findall('^(acc|jmp|nop)\s(\+|\-)(\d+)$',line.strip())[0]) for line in f.readlines()]

class track:
    def __init__(self):
        self.instructions = instructions.copy()
        self.index_dict = {index:0 for index in range(len(self.instructions))}
        self.curr_pos = -1
        self.acc_score = 0
        self.first_loop = True #part 1
        self.acc_score_before_first_loop = 0 #part 1
        self.success = False #success => last instruction is not jmp and it has been reached
        self.already_toggled_indices = []
        self.confirmed_order = []
        self.jumps_or_nops_encountered = []
        self.acc_score_before_anomaly = 0 #part2
        self.toggle_count=0
    def check_success(self):
        #return ((self.curr_pos==len(self.instructions)-1) and (self.instructions[self.curr_pos][0]!='jmp'))
        return (self.curr_pos==len(self.instructions)-1) 
    def toggle_instruction(self, instruction):
        toggle = {'jmp':'nop', 'nop':'jmp'}
        toggled_instruction = ([toggle[instruction[0]]]+instruction[1:] if instruction[0] != 'acc' else instruction)
        return toggled_instruction
    def change_curr_pos(self):
        increment = (1 if self.instructions[self.curr_pos][0]!='jmp' else (int(self.instructions[self.curr_pos][2]) if self.instructions[self.curr_pos][1]=='+' else -1*int(self.instructions[self.curr_pos][2])))
        #self.curr_pos = (self.curr_pos+increment)%len(self.instructions)
        self.curr_pos = (self.curr_pos+increment)
    def change_acc_score(self):
        self.acc_score+=((int(self.instructions[self.curr_pos][2]) if self.instructions[self.curr_pos][1]=='+' else -1*int(self.instructions[self.curr_pos][2])) if self.instructions[self.curr_pos][0]=='acc' else 0)
    def identify_loop_or_end(self):
        if self.curr_pos == -1:
            self.curr_pos = 0
        else:
            self.change_curr_pos()
        #Check for success
        if self.check_success():
            self.index_dict[self.curr_pos] = 1
            self.change_acc_score()
            self.success=True
            self.confirmed_order.append(self.curr_pos)
            self.instructions[self.already_toggled_indices[-1]]=self.toggle_instruction(self.instructions[self.already_toggled_indices[-1]])#Revert the toggle
            return ('s', self.acc_score)
        #check for loop
        elif self.index_dict[self.curr_pos]!=0:
            if self.first_loop:
                self.acc_score_before_first_loop=self.acc_score
                self.first_loop=False
            return ('l', self.acc_score)
        #Neither end nor loop
        else:
            self.index_dict[self.curr_pos] = 1
            self.change_acc_score()
            self.confirmed_order.append(self.curr_pos)
            if (self.instructions[self.curr_pos][0]!='acc') and  self.first_loop:
                self.jumps_or_nops_encountered.append(self.curr_pos)
            return self.identify_loop_or_end()
    def find_the_error(self):
        while not self.success:
            status, _ = self.identify_loop_or_end()
            if status=='l':
                if self.already_toggled_indices:
                    self.instructions[self.already_toggled_indices[-1]]=self.toggle_instruction(self.instructions[self.already_toggled_indices[-1]])#Revert the toggle
                last_untoggled_jmp_nop = self.jumps_or_nops_encountered.pop(-1)
                self.toggle_count+=1
                self.already_toggled_indices.append(last_untoggled_jmp_nop)
                for index in self.confirmed_order[self.confirmed_order.index(last_untoggled_jmp_nop):]:
                    self.acc_score-=((int(self.instructions[index][2]) if self.instructions[index][1]=='+' else -1*int(self.instructions[index][2])) if self.instructions[index][0]=='acc' else 0)
                    self.index_dict[index]=0
                self.acc_score_before_anomaly = self.acc_score
                del self.confirmed_order[self.confirmed_order.index(last_untoggled_jmp_nop):]
                self.curr_pos = self.confirmed_order[-1]
                self.instructions[last_untoggled_jmp_nop]=self.toggle_instruction(self.instructions[last_untoggled_jmp_nop])
        return self.acc_score
    
execution = track()
#part1 - 1814
execution.identify_loop_or_end()[1]
#part2 - 1056
execution.find_the_error()

1814

1056